In [ ]:
import numpy as np
from scipy.linalg import cholesky
import anDiffReg as an

In [ ]:
# exemplary data: simulated fractional Brownian motion

n = 10**4 # number of trajectories
alpha, D = 0.8, 1 # FBM parameters: anomalous index α and diffusivity D
ln = 100 # trajectory length
dt = 1 # time interval
ts = dt * np.arange(1, ln+1)


def K(s, t): # FBM covariance
    return D * (t**alpha + s**alpha - np.abs(s - t)**alpha)

S = np.array([[K(s, t) for t in ts] for s in ts])
A = cholesky(S, lower=False)
xi = np.random.randn(len(ts), n)
X = A.T @ xi
xi = np.random.randn(len(ts), n) 
Y = A.T @ xi

In [ ]:
# alternatively, load data from the file

import pandas as pd
df = pd.read_csv('exemplaryData/2D FBM.csv')
X, Y = df.values[:,::2], df.values[:,1::2]

ln, n = X.shape
dt = 1 
ts = dt * np.arange(1, ln+1)

Calculating and plotting TA-MSD

In [ ]:
msd = an.tamsd(np.dstack((X,Y))) # TA-MSD of 2D traj

In [ ]:
# TA-MSD plot with pointwise 95% error bars
import matplotlib.pyplot as plt
import scipy.stats as sps

k = 42 # choose TA-MSD to plot
w = 50 # plot window size

err, logErr = an.errCov(ts, 2, alpha)
qs = [sps.norm(loc=0, scale=np.sqrt(D)*s).ppf(0.975) for s in np.sqrt(np.diagonal(err))]

plt.scatter(ts[:w], msd[:w,k],
    label = "TA-MSD with 95% pointwise error bars",
)
plt.errorbar(ts[:w], msd[:w,k], qs[:w],
    ecolor = "black"
)
plt.xlabel("time")
plt.ylabel("TA-MSD")
plt.ylim(0,None)
plt.legend()

Parameter estimation

In [ ]:
# OLS

dim = 2
ols, covOLS = an.fit_ols(msd, dim, dt) # here 10% default window is used

In [ ]:
np.cov(ols) # covariance of the estimates, compare with predicted covOLS

In [ ]:
np.mean(ols, axis = 1)  # average estimates

In [ ]:
# GLS

alpha_init = 0.8 # used only optionally for initial estimate

gls, covGLS = an.fit_gls(msd, dim, dt, np.full(10**4,alpha_init)) 

In [ ]:
np.mean(gls, axis = 1)

In [ ]:
np.cov(gls)

In [ ]:
# covariance is computed for each trajectory, could be slow if there are many

glsP, covGLSP = an.fit_gls(msd[:,:100], dim, dt, np.full(10**4,0.8), precompute = False)

In [ ]:
# scatter plot of the estimated parameters with a 95% confidence area
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

plt.scatter(gls[0,:1000], gls[1,:1000],
    label = "GLS estimated parameters",
    s = 5            
)
plt.xlabel("log₁₀D")
plt.ylabel("α")

mlogD, mA = np.mean(gls, axis = 1) 
C = sqrtm(an.cov_gls(mA, dt, ln, dim)) # from the predicted covariance of the mean estimated parameters

def fx(t): return np.sqrt(5.99) * ( C[0,0]*np.cos(t) + C[0,1]*np.sin(t) ) + mlogD
def fy(t): return np.sqrt(5.99) * ( C[1,0]*np.cos(t)+ C[1,1]*np.sin(t) ) + mA

phis = np.linspace(0,2*np.pi,200)
plt.plot(fx(phis), fy(phis),
    label = "95% confidence area",
    linestyle = "dashed",
    linewidth = 1,
    color = "black"
)
plt.legend()

In [ ]:
# trajectories with noise

D, sigma = 5, 1

X2 = np.sqrt(D)*X + sigma*np.random.randn(len(ts), n)
msd2 = an.tamsd(X2)

# initial estimates of bot alpha and D must be used, sigma must be known 
gls2, covGLS2 = an.fit_gls(msd2, 1, 1, np.full(10**4,alpha), np.full(10**4,D), sigma)

In [ ]:
np.mean(gls2, axis=1), np.cov(gls2)